In [14]:
import typing
import weave
from weave import panels

In [15]:
import os

In [16]:
# Get Ganilla style + content images
import wandb
run = wandb.init(project="ganimals")
tb = run.use_artifact('stacey/ganimals/run-3r46hvkr-samples:v0').get("samples")
#artifact_dir = artifact.download()
#print(artifact_dir)
wandb.run.finish()

gl_src = []
gl_dst = []
for row in tb.data:
    s_i = row[0].image
    t_i = row[1].image
    gl_src.append(s_i)
    gl_dst.append(t_i)

In [27]:
# This Type should be built-in to Weave, declared in weave.types
@weave.type()
class User:
    name: str

# This Type will be a built-in to Weave, declared in weave.types
@weave.type()
class TargetMetric:
    name: str
    direction: str # typing.Union['up','down']  # (TODO: enum)

class MarkdownString(weave.types.Type):
    pass

# this is hacky
class List(weave.types.Type):
    pass

#@weave.type()
#class ExampleTable():

class TypedDict(weave.types.Type):
    pass
        
@weave.type()
class ModelCard:
    model_name: str
    created_by: User
    updated_at: str  # TODO: timestamp
    model_type: str  # TODO: enum
    primary_metric: TargetMetric
    application: str
    sample_report: str
    github_url: str
    arxiv_url: str
    content_images: TypedDict
    style_images : List
    result_images: List
    limitations: str
    use: str
    
    # TODO: This is not general enough. It should depend on the type of the model
    example: MarkdownString
        
# This should be an op, so we can call it from the UI, but I need to fix something
# to make that work
#@weave.op()
def model_card_panel(model_card: ModelCard) -> panels.Card:
    # maybe I can make a little table here?
    #temp_table = weave.panels.Table(data)
    #temp_table = weave.save(data, name="dogs")
    table = weave.panels.Table(data)
    table.table_query.add_column(lambda row: row)
    #table.table_query.add_column(lambda row: add_hello(row))
    return panels.Card(
        title=model_card.model_name,
        subtitle=model_card.created_by.name,
        content=[
            panels.CardTab(
                name='Overview',
                content=panels.Group(
                    items=[
                        panels.Group(
                            prefer_horizontal=True,
                            items=[
                                panels.LabeledItem(item=model_card.updated_at, label='Last updated'),
                                panels.LabeledItem(item=model_card.model_type, label='Model type'),
                                panels.LabeledItem(item=model_card.primary_metric.name, label='Metric'),
                            ]
                        ),
                        panels.LabeledItem(item=model_card.application, label='Application')
                        # TODO can't nest an image
                        #panels.LabeledItem(item=model_card.example, label='Example'),
                    ]
                )
            ),
            panels.CardTab(
                name='Limitations & Use',
                content=panels.Group(
                    items=[
                        panels.LabeledItem(item=model_card.limitations, label='Limitations'),
                        panels.LabeledItem(item=model_card.use, label='Use')
                    ])
            ),
            panels.CardTab(
                # TODO: can this be a more complicated Table?
                name="Content Images",
                content=panels.Table(model_card.content_images)
            ),
            panels.CardTab(
                name="Style Image(s)",
                content=panels.Table(model_card.style_images)
            ),
            panels.CardTab(
                name="Sample Result",
                content=panels.Table(model_card.result_images)
            ),
            panels.CardTab(
                name='Reference',
                content=panels.Group(
                    items=[
                        panels.LabeledItem(item=model_card.github_url, label="Github Repository"),
                        panels.LabeledItem(item=model_card.arxiv_url, label="Arxiv Link"),
                        panels.LabeledItem(item=model_card.sample_report, label="Sample W&B Report")
                    ]
                )
            ),
             panels.CardTab(
                # TODO: can we nest multiple tables here?
                name='Full Example',
                content=panels.Group(
                    items=[
                        panels.Group(
                            #prefer_horizontal=True,
                            items=[
                                panels.Table(dog_images), # label="Content Images"),
                                panels.Table([Image.open("sample_images/cat/ll2_cat.png")])
                                                  # label="Style Image")
                            ]
                        ),
                        panels.LabeledItem(item=model_card.application, label='Application'),
                       # panels.LabeledItem(item=model_card.example, label='Example')
                    ]
                )
            ),

        ]
    )

In [18]:
!pip install pillow

You should consider upgrading via the '/Users/stacey/.pyenv/versions/3.9.7/envs/wint/bin/python3.9 -m pip install --upgrade pip' command.


In [19]:
# Get all dog images as Table
from PIL import Image
data = []
dogs = "sample_images/dog"
dog_paths = [os.path.join(dogs, d) for d in os.listdir(dogs) if len(d.split(".")[0])>1]
del(dog_paths[2])
del(dog_paths[2])
dog_images = [Image.open(d).resize((512, 512)) for d in dog_paths]
data.extend([{"img_path": img_path, "image": img} for (img_path, img) in zip(dog_paths, dog_images)])

In [20]:
print(data)

[{'img_path': 'sample_images/dog/nb_dog.jpg', 'image': <PIL.Image.Image image mode=RGB size=512x512 at 0x1281D05E0>}, {'img_path': 'sample_images/dog/ar_dog.jpg', 'image': <PIL.Image.Image image mode=RGB size=512x512 at 0x13ACFD370>}, {'img_path': 'sample_images/dog/ss_dog.jpg', 'image': <PIL.Image.Image image mode=RGB size=512x512 at 0x1281EFD00>}, {'img_path': 'sample_images/dog/tamora.jpg', 'image': <PIL.Image.Image image mode=RGB size=512x512 at 0x14C26BD30>}, {'img_path': 'sample_images/dog/romeo.jpg', 'image': <PIL.Image.Image image mode=RGB size=512x512 at 0x14C26B100>}, {'img_path': 'sample_images/dog/lca_dog.jpg', 'image': <PIL.Image.Image image mode=RGB size=512x512 at 0x14C26B400>}, {'img_path': 'sample_images/dog/mo_dog.jpeg', 'image': <PIL.Image.Image image mode=RGB size=512x512 at 0x14C265250>}]


In [28]:
model_card = ModelCard(
    model_name='StarGAN-V2',
    created_by=User(name='clovaai'),
    updated_at='July 20, 2020',  # TODO: timestamp
    model_type='StarGAN',
    primary_metric=TargetMetric(name='FID', direction='down'),
    application="StarGAN-v2 combines content and style reference images to generate diverse and high-quality images with more flexibility across datasets and latent domains.",
    example="TBD",
    sample_report="https://wandb.ai/stacey/stargan/reports/Cute-Animals-and-Post-Modern-Style-Transfer-StarGAN-v2-for-Multi-Domain-Image-Synthesis---VmlldzoxNzcwODQ?galleryTag=intermediate",
    github_url="https://github.com/clovaai/stargan-v2",
    arxiv_url="https://arxiv.org/abs/1912.01865",
    limitations="Requires a pretrained model on a fixed number of N domains—these can be implicit and results may vary based on the size of N, the number of reference images in each input domain, and their diversity.",
    use="python stargan-v2/main.py --mode custom -s [content image filename] -r [reference or style image filename] -o [output file name]",
    content_images=data,
    style_images = [Image.open("sample_images/cat/ll2_cat.png")],
    result_images=[Image.open("sample_ref_stargan.jpg")] #starganv2_cross.jpg")]
)

# Render it using the model_card_panel!
model_card_panel(model_card)

In [19]:
# load ganilla style images
art_ims = []
sample_dir = "ganilla/data/illustrator_samples"
art_paths = [os.path.join(sample_dir, d) for d in os.listdir(sample_dir) if len(d.split(".")[0])>1]
art_images = [Image.open(d).resize((700, 225)) for d in art_paths]
art_ims.extend([{"img_path": img_path, "image": img} for (img_path, img) in zip(art_paths, art_images)])
print(art_paths)

['ganilla/data/illustrator_samples/KH_illustrator_sample.png', 'ganilla/data/illustrator_samples/AS_illustrator_sample.png', 'ganilla/data/illustrator_samples/DM_illustrator_sample.png', 'ganilla/data/illustrator_samples/KP_illustrator_sample.png']


In [20]:
model_card = ModelCard(
    model_name='GANILLA',
    created_by=User(name='giddyyupp'),
    updated_at='July 20, 2020',  # TODO: timestamp
    model_type='ganilla',
    primary_metric=TargetMetric(name='Style&ContentCNN', direction='up'),
    application="translate content from one image into the style of an illustrator using a pretrained network",
    example="TBD",
    github_url="https://github.com/giddyyupp/ganilla",
    arxiv_url="https://arxiv.org/abs/2002.05638",
    sample_report="https://wandb.ai/stacey/ganilla/reports/GANILLA--Vmlldzo2NTQ1NA",
    limitations="Each GANILLA model variant is finetuned on one illustrator or artist. The number, quality, and diversity of representative artist images avaialble for pretraining will affect the result quality. Model is best suited for natural images like photos of landscapes or faces and may not perform as well on other image types",
    use="!python ganilla/test.py --dataroot sample_images/dog --from_artifact ganilla_KP_pretrained:v0 --model test --log_all --gpu_ids -1",
    content_images=gl_src,
    result_images=gl_dst,
    style_images=art_images
)

# Render it using the model_card_panel!
model_card_panel(model_card)